# Covertype Data Set Preprocessing

In [ ]:
%pip install gcsfs

In [1]:
import numpy as np
import pandas as pd
import tensorflow_data_validation as tfdv

from sklearn.model_selection import train_test_split

### Set the paths

In [15]:
FULL_DATASET = '../covertype.csv'
TRAINING_DATASET='../covertype_training.csv'
TRAINING_DATASET_WITH_MISSING = '../covertype_training_missing.csv'
EVALUATION_DATASET='../covertype_evaluation.csv'
EVALUATION_DATASET_WITH_ANOMALIES='../covertype_evaluation_anomalies.csv'
SERVING_DATASET='../covertype_serving.csv'

ORIGINAL_DATASET_PATH = 'gs://workshop-datasets/covertype/orig/covtype.data'

### Load the dataset

In [3]:
df = pd.read_csv(ORIGINAL_DATASET_PATH, header=None)
print(df.shape)
df.head()

(581012, 55)


,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


### Configure soil type and wilderness area domains

In [4]:
soil_type = [
"1", "2702", "Cathedral family - Rock outcrop complex, extremely stony.",
"2", "2703", "Vanet - Ratake families complex, very stony.",
"3", "2704", "Haploborolis - Rock outcrop complex, rubbly.",
"4", "2705", "Ratake family - Rock outcrop complex, rubbly.",
"5", "2706", "Vanet family - Rock outcrop complex complex, rubbly.",
"6", "2717", "Vanet - Wetmore families - Rock outcrop complex, stony.",
"7", "3501", "Gothic family.",
"8", "3502", "Supervisor - Limber families complex.",
"9", "4201", "Troutville family, very stony.",
"10", "4703", "Bullwark - Catamount families - Rock outcrop complex, rubbly.",
"11", "4704", "Bullwark - Catamount families - Rock land complex, rubbly.",
"12", "4744", "Legault family - Rock land complex, stony.",
"13", "4758", "Catamount family - Rock land - Bullwark family complex, rubbly.",
"14", "5101", "Pachic Argiborolis - Aquolis complex.",
"15", "5151", "unspecified in the USFS Soil and ELU Survey.",
"16", "6101", "Cryaquolis - Cryoborolis complex.",
"17", "6102", "Gateview family - Cryaquolis complex.",
"18", "6731", "Rogert family, very stony.",
"19", "7101", "Typic Cryaquolis - Borohemists complex.",
"20", "7102", "Typic Cryaquepts - Typic Cryaquolls complex.",
"21", "7103", "Typic Cryaquolls - Leighcan family, till substratum complex.",
"22", "7201", "Leighcan family, till substratum, extremely bouldery.",
"23", "7202", "Leighcan family, till substratum - Typic Cryaquolls complex.",
"24", "7700", "Leighcan family, extremely stony.",
"25", "7701", "Leighcan family, warm, extremely stony.",
"26", "7702", "Granile - Catamount families complex, very stony.",
"27", "7709", "Leighcan family, warm - Rock outcrop complex, extremely stony.",
"28", "7710", "Leighcan family - Rock outcrop complex, extremely stony.",
"29", "7745", "Como - Legault families complex, extremely stony.",
"30", "7746", "Como family - Rock land - Legault family complex, extremely stony.",
"31", "7755", "Leighcan - Catamount families complex, extremely stony.",
"32", "7756", "Catamount family - Rock outcrop - Leighcan family complex, extremely stony.",
"33", "7757", "Leighcan - Catamount families - Rock outcrop complex, extremely stony.",
"34", "7790", "Cryorthents - Rock land complex, extremely stony.",
"35", "8703", "Cryumbrepts - Rock outcrop - Cryaquepts complex.",
"36", "8707", "Bross family - Rock land - Cryumbrepts complex, extremely stony.",
"37", "8708", "Rock outcrop - Cryumbrepts - Cryorthents complex, extremely stony.",
"38", "8771", "Leighcan - Moran families - Cryaquolls complex, extremely stony.",
"39", "8772", "Moran family - Cryorthents - Leighcan family complex, extremely stony.",
"40", "8776", "Moran family - Cryorthents - Rock land complex, extremely stony.",
]

wilderness_area = [
"Rawah", "Rawah Wilderness Area",
"Neota", "Neota Wilderness Area",
"Commanche", "Comanche Peak Wilderness Area",
"Cache", "Cache la Poudre Wilderness Area"
]

### Map one-hot encoded values to categorical domains

In [5]:
soil = df.loc[:, 14:53].apply(lambda x: soil_type[1::3][x.to_numpy().nonzero()[0][0]], axis=1)
soil

0         7745
1         7745
2         4744
3         7746
4         7745
          ... 
581007    2703
581008    2703
581009    2703
581010    2703
581011    2703
Length: 581012, dtype: object

In [6]:
wilderness = df.loc[:, 10:13].apply(lambda x: wilderness_area[0::2][x.to_numpy().nonzero()[0][0]], axis=1)
wilderness

0             Rawah
1             Rawah
2             Rawah
3             Rawah
4             Rawah
            ...    
581007    Commanche
581008    Commanche
581009    Commanche
581010    Commanche
581011    Commanche
Length: 581012, dtype: object

### Create a dataset with column names and categorical values replacing one-hot encoded soil type and wilderness areas

In [7]:
COLUMN_NAMES = [
    'Elevation', 
    'Aspect', 
    'Slope', 
    'Horizontal_Distance_To_Hydrology',
    'Vertical_Distance_To_Hydrology',
    'Horizontal_Distance_To_Roadways',
    'Hillshade_9am',
    'Hillshade_Noon',
    'Hillshade_3pm',
    'Horizontal_Distance_To_Fire_Points',
    'Wilderness_Area',
    'Soil_Type',
    'Cover_Type']

df_full = pd.concat([df.loc[:, 0:9], wilderness, soil, df.loc[:, 54]], axis=1, ignore_index=True)
df_full.columns = COLUMN_NAMES
df_full

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,Rawah,7745,5
1,2590,56,2,212,-6,390,220,235,151,6225,Rawah,7745,5
2,2804,139,9,268,65,3180,234,238,135,6121,Rawah,4744,2
3,2785,155,18,242,118,3090,238,238,122,6211,Rawah,7746,2
4,2595,45,2,153,-1,391,220,234,150,6172,Rawah,7745,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,Commanche,2703,3
581008,2391,152,19,67,12,95,240,237,119,845,Commanche,2703,3
581009,2386,159,17,60,7,90,236,241,130,854,Commanche,2703,3
581010,2384,170,15,60,5,90,230,245,143,864,Commanche,2703,3


### Save the dataset to CSV file

In [8]:
df_full.to_csv(FULL_DATASET, header=True, index=False)

### Create training, validation, testing and serving splits.

In [23]:
df_full = df = pd.read_csv(FULL_DATASET)
df_full

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,Rawah,7745,5
1,2590,56,2,212,-6,390,220,235,151,6225,Rawah,7745,5
2,2804,139,9,268,65,3180,234,238,135,6121,Rawah,4744,2
3,2785,155,18,242,118,3090,238,238,122,6211,Rawah,7746,2
4,2595,45,2,153,-1,391,220,234,150,6172,Rawah,7745,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,Commanche,2703,3
581008,2391,152,19,67,12,95,240,237,119,845,Commanche,2703,3
581009,2386,159,17,60,7,90,236,241,130,854,Commanche,2703,3
581010,2384,170,15,60,5,90,230,245,143,864,Commanche,2703,3


In [24]:
df_train, df_other = train_test_split(df_full, train_size=431012, shuffle=True)
df_evaluate = df_other.iloc[0:75000]
df_serving = df_other.iloc[75000:150000]
df_serving = df_serving.drop(columns=['Cover_Type'])
print(df_train.shape)
print(df_evaluate.shape)
print(df_serving.shape)

(431012, 13)
(75000, 13)
(75000, 12)


Add some missing values to the training split.

In [41]:
df_train_missing = df_train.reset_index(drop=True)
df_train_missing.loc[0:8999, 'Horizontal_Distance_To_Hydrology'] = None
df_train_missing

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,3061,235,6,NaN,7,2766,210,245,173,579,Commanche,7202,1
1,2811,312,15,NaN,6,2496,179,228,189,2584,Rawah,7746,1
2,2826,335,11,NaN,19,1423,195,225,169,1972,Commanche,7755,6
3,3029,32,8,NaN,7,1077,219,224,141,698,Commanche,7756,2
4,2693,68,14,NaN,76,2391,234,210,105,883,Commanche,2703,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
431007,3132,0,5,510.0,-42,4018,213,231,156,4375,Rawah,7202,1
431008,3184,76,4,210.0,11,5408,225,232,143,2763,Rawah,7202,1
431009,2816,129,23,120.0,65,2100,250,219,84,811,Commanche,4758,5
431010,2817,81,5,30.0,-1,1909,226,231,140,2505,Commanche,7702,2


Create the evaluation split where some values of Slope are more than 90 degrees and some examples have 2701 code for soil type (2701 is not present in the training split)

In [42]:
df_evaluate_anomalies = df_evaluate.reset_index(drop=True)
df_evaluate_anomalies.loc[0:4, 'Slope'] = 110
df_evaluate_anomalies.loc[5:7, 'Soil_Type'] = 2701
df_evaluate_anomalies

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,3051,7,110,0,0,2568,211,225,151,778,Commanche,7202,1
1,2820,212,110,127,22,3679,205,253,180,3237,Rawah,7746,2
2,2288,2,110,134,18,626,165,170,130,601,Cache,4703,6
3,3155,31,110,256,11,3079,216,210,128,3120,Commanche,7700,2
4,2945,90,110,127,13,3637,237,223,116,6449,Rawah,7745,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74995,3225,31,6,242,22,1800,219,228,145,1332,Commanche,7755,1
74996,3025,41,10,90,-1,3290,221,218,131,1471,Commanche,7202,1
74997,2862,53,10,95,7,3227,226,219,127,2430,Rawah,7202,1
74998,3094,115,20,420,39,552,250,216,84,1266,Commanche,7756,1


Save the splits to local files.

In [43]:
#df_train.to_csv(TRAINING_DATASET, header=True, index=False)
df_train_missing.to_csv(TRAINING_DATASET_WITH_MISSING, header=True, index=False)
#df_evaluate.to_csv(EVALUATION_DATASET, header=True, index=False)
#df_evaluate_anomalies.to_csv(EVALUATION_DATASET_WITH_ANOMALIES, header=True, index=False)
#df_serving.to_csv(SERVING_DATASET, header=True, index=False)

Copy the splits to GCS

In [44]:
#!gsutil cp $TRAINING_DATASET gs://workshop-datasets/covertype/training/
!gsutil cp $TRAINING_DATASET_WITH_MISSING gs://workshop-datasets/covertype/training_missing/
#!gsutil cp $EVALUATION_DATASET gs://workshop-datasets/covertype/evaluation/
#!gsutil cp $EVALUATION_DATASET_WITH_ANOMALIES gs://workshop-datasets/covertype/evaluation_anomalies/
#!gsutil cp $SERVING_DATASET gs://workshop-datasets/covertype/serving/

Copying file://../covertype_training_missing.csv [Content-Type=text/csv]...
- [1 files][ 23.0 MiB/ 23.0 MiB]                                                
Operation completed over 1 objects/23.0 MiB.                                     
